In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: 17
### Team Member Names: Nelson Kang, Mane Muradyan
### Team Strategy Chosen: Market Beat

Goal: Market beat
Strategy: Sortino-centered stock selection with some short-term momentum tilt

Inputs:
- CSV file with tickers

Outputs:
- 

Game Plan:
1. load csv and set constants
2. get FX rate and static info of stocks
3. check eligibility of stocks (liquidity rule etc.)
4. Compute Sortino and momentum tilt and combine for a score
5. construct portfolio with cap and sector mix
6. Distribute stock weight


In [ ]:
#Imports
import math
import time
import random
import numpy as np
import pandas as pd
import yfinance as yf
import os


# Defining most constants
N_min = 10 # min number of stocks in portfolio
N_max = 25 # max number of stocks in portfolio
max_weight = 0.15 # max weight per stock
sector_cap = 0.4 # max weight per sector
budget = 1_000_000 
MIN_avg_daily_volume = 5000
LIVE_date = "2025-11-21"
END_date = "2025-11-28"

fee_flat_usd = 2.15
fee_per_share_usd = 0.001


LOOK_BACK_START = "2022-01-01"
LOOK_BACK_END = "2025-08-08"



In [ ]:
#HELPERS
def get_tickers(file_name):
    """
    Input: local name of tickers csv file
    Output: list of tickers
    """
    df = pd.read_csv(file_name)
    first_col = df.columns[0]
    tickers = (df[first_col].str.strip().tolist())
    return tickers

def get_info(tickers_lst):
    """
    Input: list of tickers
    Output: dict: {ticker: {'sector': str, 'marketCap': float, 'currency': str}} 
    Function fetches basic info for list of tickers
    """
    output = {}
    for ticker in tickers_lst:
        try:
            t = yf.Ticker(ticker)
            info = t.get_info()
            sector = info.get("industry")
            mcap = info.get("marketCap")
            currency = info.get("currency")
            output[t] = {
                        'ticker' : ticker,
                        'sector' : sector,
                         'MarketCap' : mcap,
                         'Currency' : currency}
        except Exception as e:
            output[t] = {'ticker' : None,
                        'sector' : None,
                         'MarketCap' : None,
                         'Currency' : None}
    return output

def get_exchange_rate():
    """
    Input: None
    Output: Float
    Returns latest exchange rate CAD --> USD
    """
    fx = yf.Ticker("CADUSD=X")
    return float(fx.fast_info["last_price"])


def weekly_closing(ticker_lst, start, end):
    full_data = yf.download(ticker_lst, start, end, progress=False, auto_adjust=True)["Close"]
    return full_data.resample("W").last()


def get_history(ticker_lst, start, end):
    """
    
    """
    data = yf.download(ticker_lst, start, end, interval='1d', auto_adjust=False, progress=False)
    prices = data['Adj Close'].copy()
    vols = data["Volume"].copy()
    return prices, vols

def monthly_filter_avg_volume(vol_series):
    return 

def get_risk_free_rate():
    """
    Gets Canadian risk free rate using Tbills 
    """
    ticker = yf.Ticker("^IRX.CA") # canadian tbill
    data = ticker.history(period='1d')["Close"].iloc[-1]
    return (data / 100)

def get_sharpe_ratio(t):
    risk_free_rate = get_risk_free_rate() # closest to US treasury bills 2024-2025, might want to make a function to auto calc it
    data = yf.download(t, start=LOOK_BACK_START, end=LOOK_BACK_END, progress=False)["Adj Close"]
    daily_returns = data.pct_change().dropna()
    annual_return = (1 + daily_returns.mean())**252 - 1
    annual_volatility = daily_returns.std() * np.sqrt(252)
    sharpe = (annual_return - risk_free_rate) / annual_volatility
    return

In [ ]:
#PORTFOLIO CONSTRUCTION
def possible_companies(ticker_lst, FX):

    all_prices, Volume_all = get_history(ticker_lst, "2024-10-01", "2025-09-30")

    pos_companies = {}

    for t in ticker_lst:
        try: # filter out uneligible stocks
            # check for average daily volume to meet rule requirements
            vol_ser = Volume_all.get(t)
            avg_volume = monthly_filter_avg_volume(vol_ser)
            last_close = float()
            if avg_volume < MIN_avg_daily_volume:
                continue


            t_obj = yf.Ticker(t)   
            info = t_obj.get_info()
            sector = info.get("industry")
            mcap = info.get("marketCap")
            currency = info.get("currency")
            last_close = float(t_obj.fast_info["last_price"])
            # Add sortino here
            # Add sharpe here
            sharpe = get_sharpe_ratio(t)
            # Add Max Drawdown here
            # Add momentum here

            pos_companies[t] = {
                'Ticker' : t,
                'Currency' : currency,
                'Sector' : sector,
                'Market Cap (cad)' : mcap,
                'Last Price' : last_close,
                # Add the data analysis here
            }

        except Exception:
            continue # in case a step errors

    return pos_companies


In [ ]:
def build_port(ticker_lst):
    FX = get_exchange_rate()

    candidates = possible_companies(ticker_lst, FX)
    return


#tech_data =yf.download(ticker_list, start, end, progress=False, auto_adjust=True)["Close"]

t_list = get_tickers("Tickers_file.csv")
print(t_list)
#info_dict = get_info(t_list)
#df1 = pd.DataFrame(info_dict)
#weekly_df = weekly_closing(t_list, "2022-01-01", "2025-08-08")
prices_all, vols_all = get_history(t_list, "2022-01-01", "2025-08-08")
print(vols_all)
print(prices_all)
ex = get_exchange_rate()
print(ex)


['ABBV', 'ABT', 'ACN', 'AGN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BB.TO', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CELG', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTN', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']



4 Failed downloads:
['RTN', 'MON', 'CELG', 'AGN']: YFTzMissingError('possibly delisted; no timezone found')


Ticker           ABBV        ABT         ACN  AGN        AIG         AMZN  \
Date                                                                        
2022-01-03  6839800.0  6688100.0   2129900.0  NaN  3376300.0   63520000.0   
2022-01-04  6298300.0  8241200.0   2516300.0  NaN  5281300.0   70726000.0   
2022-01-05  7724900.0  5948200.0   2471400.0  NaN  4639400.0   64302000.0   
2022-01-06  4667000.0  5710200.0   4386600.0  NaN  6331600.0   51958000.0   
2022-01-07  8630300.0  4367500.0   3469000.0  NaN  5318900.0   46606000.0   
...               ...        ...         ...  ...        ...          ...   
2025-08-01  8082000.0  5453300.0   6042400.0  NaN  4438100.0  122258800.0   
2025-08-04  4217500.0  5671400.0   3470600.0  NaN  4757900.0   77890100.0   
2025-08-05  4153400.0  5355400.0  10175900.0  NaN  2788800.0   51505100.0   
2025-08-06  4057200.0  5170100.0   8064700.0  NaN  4238500.0   54823000.0   
2025-08-07  4683300.0  4690600.0   7246300.0  NaN  5966900.0   40603500.0   

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Nelson Kang
